In [16]:
import json
import geopandas as gpd
from shapely.geometry import MultiPoint
import random
import folium
import folium.plugins as plugins
from shapely.geometry import MultiPoint
import pandas as pd
from shapely.geometry import Point
from folium import TileLayer

In [11]:
# Langkah 1: Baca file CSV menggunakan pandas
df = pd.read_csv('../data/koordinat_masjid.csv')

# Langkah 2: Konversi DataFrame ke GeoDataFrame
# Misalkan kolom latitude dan longitude di CSV bernama 'lat' dan 'long'
geometry = [Point(xy) for xy in zip(df['long'], df['lat'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry)
gdf.crs = 'EPSG:4326'
# gdf['centroid_latitude'] = gdf['geometry'].centroid.y
# gdf['centroid_longitude'] = gdf['geometry'].centroid.x
# gdf['geometry'].centroid.to_list()
gdf

,nama,lat,long,geometry
0,Al Ikhlas Bone,0.33756,123.46097,POINT (123.46097 0.33756)
1,Al Ihsan Bone Raya,0.31044,123.34598,POINT (123.34598 0.31044)
2,At Taibah Bulawa,0.32442,123.27561,POINT (123.27561 0.32442)
3,Al Arqam Suwawa Timur,0.51472,123.22386,POINT (123.22386 0.51472)
4,Nurul Ihklas Bone Pantai,0.38017,123.21278,POINT (123.21278 0.38017)
...,...,...,...,...
78,Salafiyah Syafi’iyah Taluditi,0.54806,121.79000,POINT (121.79 0.54806)
79,Arrayan Wanggarasi,0.53619,121.70239,POINT (121.70239 0.53619)
80,Al Musyawwarah Lemito,0.54458,121.57389,POINT (121.57389 0.54458)
81,An-Nuur Popayato Timur,0.51033,121.47467,POINT (121.47467 0.51033)


# Folium

In [33]:
# Titik koordinat Ka'bah
kabah_coords = [21.4225, 39.8262]

# Ambil titik-titik dari kolom 'geometry'
titik_titik = gdf['geometry'].centroid.to_list()
multi_point = MultiPoint(titik_titik)
titik_tengah = multi_point.centroid

# Function to generate a random color
def random_color():
    return '#{:06x}'.format(random.randint(0, 0xFFFFFF))

# Create a dictionary mapping each KECEMATAN to a random color
unique_distrik = gdf['nama'].unique()
color_dict = {distrik: random_color() for distrik in unique_distrik}

# Buat peta folium
m = folium.Map(location=[titik_tengah.y, titik_tengah.x], zoom_start=9, tiles='openstreetmap')
# m.options['maxZoom'] = 100  # Ganti dengan nilai maksimum yang Anda inginkan

titik_group = folium.FeatureGroup("Titik Masjid", show=True).add_to(m)
# Tambahkan label kampung pada setiap shape
for idx, row in gdf.iterrows():
    folium.Marker(
        location=[row['lat'], row['long']],
        tooltip=folium.Tooltip(  # Change Popup to Tooltip
            f"""
                <b>Nama masjid:</b> {row['nama']}<br>
                <b>Lat:</b> {row['lat']}<br>
                <b>Long:</b> {row['long']}<br>
                <b>color:</b> {color_dict[row['nama']]}<br>
            """,
            sticky=False
        ),
        # icon=folium.Icon(color='blue', icon='info-sign')
        icon=folium.Icon(color='blue', icon='mosque', prefix='fa')
        # icon=folium.DivIcon(html=f"""<div style="font-size: 10pt; text-shadow: 1px 1px 1px #ffffff; color: black">{row['nama']}</div>"""),
    ).add_to(titik_group)

    # Menambahkan garis lurus dari masjid ke Ka'bah
    folium.PolyLine(
        locations=[[row['lat'], row['long']], kabah_coords],
        color='red',
        weight=2,
        dash_array='5, 5'  # Menjadikan garis putus-putus, opsional
    ).add_to(titik_group)

plugins.Fullscreen(
    position="topright",
    title="Expand me",
    title_cancel="Exit me",
    force_separate_button=True,
).add_to(m)

# tile
folium.TileLayer("esri-worldimagery").add_to(m)
folium.TileLayer("cartodbpositron").add_to(m)
# Define the Google Maps URL template
google_maps_url = 'http://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}'
# Add the Google Maps tile layer to the map
google_maps_layer = TileLayer(tiles=google_maps_url, attr='Google', name='Google Maps')
google_maps_layer.add_to(m)
folium.LayerControl().add_to(m)

/var/folders/b9/v73gyxys1djbgjy3c8c3cfwm0000gn/T/ipykernel_27836/214657217.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  titik_titik = gdf['geometry'].centroid.to_list()


In [34]:
# Tampilkan peta
m.save('../map-html/peta.html')
# m